In [22]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
%ls

EmotionDetectionDeep.h5    fer2013.bib  README
EmotionDetectionDeep.json  fer2013.csv


In [24]:
%cd 'drive'
%ls 

[Errno 2] No such file or directory: 'drive'
/content/drive/My Drive/fer2013
EmotionDetectionDeep.h5    fer2013.bib  README
EmotionDetectionDeep.json  fer2013.csv


In [25]:
%cd 'My Drive'
%ls

[Errno 2] No such file or directory: 'My Drive'
/content/drive/My Drive/fer2013
EmotionDetectionDeep.h5    fer2013.bib  README
EmotionDetectionDeep.json  fer2013.csv


In [26]:
%cd fer2013

[Errno 2] No such file or directory: 'fer2013'
/content/drive/My Drive/fer2013


In [27]:
'''importing all the necessary libraries'''
import sys, os
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils


In [18]:
'''read the file in a pandas dataframe'''
df=pd.read_csv('fer2013.csv')

In [31]:
'''make lists for training and testing sets'''
X_train,Y_train,X_test,Y_test=[],[],[],[]

'''our initial pixel array has a type of object convert that to float32 and convert each row appending to training and testing lists to a numpy array'''
for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           Y_train.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           Y_test.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")


num_labels = 7
batch_size = 64
epochs = 30
width, height = 48, 48

'''convert the whole training and testing list into a numpy array'''
X_train = np.array(X_train,'float32')
Y_train = np.array(Y_train,'float32')
X_test = np.array(X_test,'float32')
Y_test = np.array(Y_test,'float32')

'''categorical_crossentropy requires the data in categorical form'''
Y_train=np_utils.to_categorical(Y_train, num_classes=num_labels)
Y_test=np_utils.to_categorical(Y_test, num_classes=num_labels)

'''Normalize the X inputs'''
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

'''Convert the X inputs into required form for a Neural Network'''
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)


In [32]:
print("X_train:{}".format(X_train[0:2]))
print("X_test:{}".format(X_test[0:2]))
print("Y_train:{}".format(Y_train[0:2]))
print("Y_test:{}".format(X_test[0:2]))

X_train:[[[[-0.6098866 ]
   [-0.4592209 ]
   [-0.40325198]
   ...
   [-0.7694696 ]
   [-0.90518403]
   [-0.95160526]]

  [[-0.66049284]
   [-0.68162924]
   [-0.694159  ]
   ...
   [-0.7112763 ]
   [-0.7862608 ]
   [-0.90819967]]

  [[-0.83170736]
   [-0.8952668 ]
   [-0.73603356]
   ...
   [-0.7924775 ]
   [-0.7261497 ]
   [-0.8613453 ]]

  ...

  [[-0.33269772]
   [-0.6514918 ]
   [-0.93670547]
   ...
   [-0.5265123 ]
   [-0.7377333 ]
   [-0.9078971 ]]

  [[-0.5099617 ]
   [-0.43692893]
   [-0.46537215]
   ...
   [-0.10019822]
   [-0.5580577 ]
   [-0.86989194]]

  [[-0.5095401 ]
   [-0.56317693]
   [-0.3986179 ]
   ...
   [-0.08629682]
   [-0.0573744 ]
   [-0.41127437]]]


 [[[ 0.37236458]
   [ 0.40461555]
   [ 0.41102034]
   ...
   [ 0.19833209]
   [ 0.2950228 ]
   [ 0.01161678]]

  [[ 0.3893324 ]
   [ 0.4109821 ]
   [ 0.45377484]
   ...
   [ 0.12469399]
   [ 0.3214692 ]
   [ 0.2319735 ]]

  [[ 0.40903383]
   [ 0.4540902 ]
   [ 0.56116307]
   ...
   [-0.02652   ]
   [ 0.11207935]
   

In [33]:
'''Define the model architecture '''
layers = [
          Conv2D(filters=16, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu',input_shape=(width,height,1)),
          BatchNormalization(),
          MaxPooling2D(pool_size=(2,2),strides=(2,2)),
          Conv2D(filters=32, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu'),
          BatchNormalization(),
          MaxPooling2D(pool_size=(2,2),strides=(2,2)),
          Conv2D(filters=64, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu'),
          BatchNormalization(),
          MaxPooling2D(pool_size=(2,2),strides=(2,2)),
          Conv2D(filters=128, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu'),
          BatchNormalization(),
          MaxPooling2D(pool_size=(2,2),strides=(2,2)),
          Conv2D(filters=256, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu'),
          BatchNormalization(),
          MaxPooling2D(pool_size=(2,2),strides=(2,2)),
          Flatten(),
          Dense(128,activation='relu'),
          Dense(64,activation='relu'),
          Dense(7,activation='softmax') ]

model = Sequential(layers)

model.summary()

'''Compile the model'''
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

'''Fit the model'''
model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, Y_test),
          shuffle=True)


'''Save the model weights into a h5 file'''
save_json = model.to_json()
with open('EmotionDetectionDeep.json','w') as json_file:
    json_file.write(save_json )
model.save('EmotionDetectionDeep.h5')

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 48, 48, 16)        160       
_________________________________________________________________
batch_normalization_30 (Batc (None, 48, 48, 16)        64        
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 24, 24, 32)        4640      
_________________________________________________________________
batch_normalization_31 (Batc (None, 24, 24, 32)        128       
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 12, 12, 64)       